# 02. Window Memory (최근 N개 대화 저장)

이 노트북에서는 **Window Memory** 방식을 LangChain과 LangGraph로 구현합니다.

**Window Memory란?**
- 최근 N개의 대화만 저장
- 오래된 대화는 자동 삭제
- 토큰 비용을 일정 수준으로 유지

**학습 목표:**
- Window 기반 메모리 구현
- LangGraph에서 메시지 트리밍
- 토큰 제한 관리

---

## 1. 패키지 설치

In [ ]:
!pip install langchain langchain-anthropic langgraph langgraph-checkpoint-sqlite -q
print("설치 완료!")

설치 완료!


---

## 2. 모델 선택

Claude 4.5 모델 중 하나를 선택하세요.

In [ ]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
# MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-haiku-4-5-20251001


---

## 3. API 키 설정

In [ ]:
import os

# 방법 1: 직접 입력
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

# 방법 2: Colab Secrets
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    print("Colab Secrets에서 API 키를 불러왔습니다!")
except:
    print("직접 API 키를 입력하세요.")

Colab Secrets에서 API 키를 불러왔습니다!


---

## 4. LangChain으로 Window Memory 구현

### 4.1 기본 구성

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import InMemoryChatMessageHistory

# LLM 초기화
llm = ChatAnthropic(
    model=MODEL,
    max_tokens=500
)

# 윈도우 크기 설정 (최근 N개 대화만 유지)
WINDOW_SIZE = 4  # 최근 4개 메시지 (2턴)

# 메모리
chat_history = InMemoryChatMessageHistory()

print(f"Window Memory 설정 완료! (윈도우 크기: {WINDOW_SIZE})")

Window Memory 설정 완료! (윈도우 크기: 4)


### 4.2 Window Memory 챗봇

In [ ]:
def trim_messages_to_window(messages: list, window_size: int) -> list:
    """
    메시지를 윈도우 크기로 제한합니다.
    """
    if len(messages) <= window_size:
        return messages
    return messages[-window_size:]


def chat_with_window_memory(user_input: str) -> str:
    """
    Window Memory 방식 챗봇.
    최근 WINDOW_SIZE개의 메시지만 유지합니다.
    """
    # 사용자 메시지 추가
    chat_history.add_user_message(user_input)

    # 윈도우 크기로 메시지 제한
    recent_messages = trim_messages_to_window(
        chat_history.messages,
        WINDOW_SIZE
    )

    # 시스템 메시지 + 최근 대화만
    messages = [
        SystemMessage(content="당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."),
        *recent_messages
    ]

    # LLM 호출
    response = llm.invoke(messages)

    # AI 응답 저장
    chat_history.add_ai_message(response.content)

    return response.content


print("="*50)
print("Window Memory 테스트 (윈도우 크기: 4)")
print("="*50)

Window Memory 테스트 (윈도우 크기: 4)


### 4.3 테스트: 오래된 대화는 잊어버림

In [ ]:
# 대화 1 - 이름 알려주기
response1 = chat_with_window_memory("안녕! 내 이름은 철수야.")
print(f"\n[1] 사용자: 안녕! 내 이름은 철수야.")
print(f"    AI: {response1}")

# 대화 2
response2 = chat_with_window_memory("좋아하는 색깔은 파란색이야.")
print(f"\n[2] 사용자: 좋아하는 색깔은 파란색이야.")
print(f"    AI: {response2}")

# 대화 3
response3 = chat_with_window_memory("좋아하는 음식은 피자야.")
print(f"\n[3] 사용자: 좋아하는 음식은 피자야.")
print(f"    AI: {response3}")

# 대화 4
response4 = chat_with_window_memory("취미는 게임이야.")
print(f"\n[4] 사용자: 취미는 게임이야.")
print(f"    AI: {response4}")

print("\n" + "="*50)
print("이제 오래된 대화(이름)는 윈도우 밖으로 밀려남!")
print("="*50)

# 대화 5 - 이름을 물어봄 (윈도우 밖으로 밀려남)
response5 = chat_with_window_memory("내 이름이 뭐라고 했지?")
print(f"\n[5] 사용자: 내 이름이 뭐라고 했지?")
print(f"    AI: {response5}")
print("\n[!] 이름 정보가 윈도우 밖으로 밀려나서 기억하지 못합니다.")


[1] 사용자: 안녕! 내 이름은 철수야.
    AI: 안녕하세요, 철수님! 만나서 반갑습니다. 😊

저는 당신을 도와드릴 준비가 되어 있습니다. 궁금하신 점이나 도움이 필요하신 것이 있으면 편하게 말씀해 주세요!

[2] 사용자: 좋아하는 색깔은 파란색이야.
    AI: 좋은 정보 감사합니다, 철수님! 파란색은 정말 멋진 색깔이네요. 🔵

파란색은:
- **차분하고 안정적인** 느낌을 주는 색이고
- **신뢰감과 평온함**을 상징하기도 합니다
- 많은 사람들이 선호하는 인기 있는 색깔이기도 하죠

철수님이 파란색을 좋아하시는 이유가 특별히 있을까요? 아니면 그냥 자연스럽게 끌리는 색깔인가요? 😄

[3] 사용자: 좋아하는 음식은 피자야.
    AI: 오, 피자도 정말 맛있는 음식이네요! 🍕

피자는:
- **다양한 토핑**으로 자신의 입맛에 맞게 즐길 수 있고
- **따뜻하고 풍미 있는** 음식이라 많은 사람들이 좋아하죠
- 친구들과 **함께 나눠 먹기 좋은** 음식이기도 합니다

철수님이 좋아하는 피자 종류가 있을까요? 예를 들면:
- 페퍼로니 피자?
- 하와이안 피자?
- 치즈 피자?
- 아니면 다른 종류?

그리고 파란색을 좋아하고 피자를 즐겨 드신다니, 철수님의 취향이 점점 더 궁금해지네요! 😊

[4] 사용자: 취미는 게임이야.
    AI: 오, 게임이 취미라니 멋진데요! 🎮

게임은:
- **재미있고 긴장감 있는** 경험을 주고
- **전략을 세우고 생각하는** 능력을 키울 수 있으며
- **스트레스 해소**에도 좋은 활동이죠

철수님이 즐기시는 게임의 종류가 궁금한데요:
- **액션 게임**을 좋아하시나요?
- **RPG(역할극 게임)**는요?
- **전략 게임**?
- **스포츠 게임**?
- 아니면 다른 종류의 게임?

그리고 어떤 플랫폼에서 주로 게임을 즐기세요?
- PC 게임?
- 콘솔 게임 (PS, Xbox 등)?
- 모바일 게임?

철수님에 대해 알아가니 정말 좋네요! 파란색, 피자, 게임... 멋진 취향들을 가지고 있으시네요

### 4.4 현재 윈도우 상태 확인

In [ ]:
print(f"전체 저장된 메시지: {len(chat_history.messages)}개")
print(f"윈도우 크기: {WINDOW_SIZE}개")
print(f"실제 사용되는 메시지: 최근 {min(len(chat_history.messages), WINDOW_SIZE)}개")
print("\n현재 윈도우 내 대화:")
print("-" * 50)

recent = trim_messages_to_window(chat_history.messages, WINDOW_SIZE)
for i, msg in enumerate(recent, 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"[{i}] {role}: {content}")

전체 저장된 메시지: 10개
윈도우 크기: 4개
실제 사용되는 메시지: 최근 4개

현재 윈도우 내 대화:
--------------------------------------------------
[1] 사용자: 취미는 게임이야.
[2] AI: 오, 게임이 취미라니 멋진데요! 🎮

게임은:
- **재미있고 긴장감 있는** 경험을 주고...
[3] 사용자: 내 이름이 뭐라고 했지?
[4] AI: 아, 죄송합니다! 😅

저는 당신의 이름을 들은 적이 없습니다. 제가 마음대로 "철수"라고...


In [ ]:
type(recent), recent

(list,
 [HumanMessage(content='취미는 게임이야.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오, 게임이 취미라니 멋진데요! 🎮\n\n게임은:\n- **재미있고 긴장감 있는** 경험을 주고\n- **전략을 세우고 생각하는** 능력을 키울 수 있으며\n- **스트레스 해소**에도 좋은 활동이죠\n\n철수님이 즐기시는 게임의 종류가 궁금한데요:\n- **액션 게임**을 좋아하시나요?\n- **RPG(역할극 게임)**는요?\n- **전략 게임**?\n- **스포츠 게임**?\n- 아니면 다른 종류의 게임?\n\n그리고 어떤 플랫폼에서 주로 게임을 즐기세요?\n- PC 게임?\n- 콘솔 게임 (PS, Xbox 등)?\n- 모바일 게임?\n\n철수님에 대해 알아가니 정말 좋네요! 파란색, 피자, 게임... 멋진 취향들을 가지고 있으시네요! 😄', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내 이름이 뭐라고 했지?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='아, 죄송합니다! 😅\n\n저는 당신의 이름을 들은 적이 없습니다. 제가 마음대로 "철수"라고 부른 것 같네요. 정말 실례했습니다!\n\n당신의 진짜 이름이 뭐예요? 알려주실래요? 😊', additional_kwargs={}, response_metadata={})])

---

## 5. LangGraph로 Window Memory 구현

LangGraph의 `trim_messages` 함수를 활용합니다.

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

WINDOW_SIZE = 6  # 최근 6개 메시지 (3턴)
WINDOW_SIZE_LANGGRAPH = 6
def windowed_add_messages(existing: list, new: list) -> list:
    """윈도우 크기를 유지하는 커스텀 reducer"""
    combined = existing + new
    # 윈도우 크기 초과 시 오래된 메시지 삭제
    if len(combined) > WINDOW_SIZE:
        return combined[-WINDOW_SIZE:]
    return combined

class State(TypedDict):
    messages: Annotated[list, windowed_add_messages]  # 커스텀 reducer

def chatbot_node(state: State) -> dict:
    messages = [
        SystemMessage(content="당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."),
        *state["messages"]
    ]
    response = llm.invoke(messages)
    return {"messages": [response]}

# 그래프 구성
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

memory_saver = MemorySaver()
graph = graph_builder.compile(checkpointer=memory_saver)

### 5.1 LangGraph Window Memory 테스트

In [ ]:
config = {"configurable": {"thread_id": "window-session-1"}}

def chat_with_langgraph_window(user_input: str) -> str:
    """LangGraph 기반 Window Memory 챗봇"""
    result = graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    return result["messages"][-1].content


print("="*50)
print(f"LangGraph Window Memory 테스트 (윈도우: {WINDOW_SIZE_LANGGRAPH})")
print("="*50)

# 여러 대화 진행
conversations = [
    "안녕!  내 이름은 민수야.",
    "스파게티 맛있다",
    "2+7 = 9.",
    "오 샹젤리제.",
    "날씨가 좋네.",
    "배가 많이 고프네",
    "죽느냐 사느냐 그것이 문제로다."
]

for i, msg in enumerate(conversations, 1):
    response = chat_with_langgraph_window(msg)
    print(f"\n[{i}] 사용자: {msg}")
    print(f"    AI: {response}")

print("\n" + "="*50)
print("오래된 정보 확인")
print("="*50)

# 오래된 정보 질문
response = chat_with_langgraph_window("내 이름이 뭐라고 했지?")
print(f"\n[6] 사용자: 이 뭐라고 했지?")
print(f"    AI: {response}")

LangGraph Window Memory 테스트 (윈도우: 6)

[1] 사용자: 안녕!  내 이름은 민수야.
    AI: 안녕하세요, 민수님! 만나서 반갑습니다. 😊

저는 여러분을 도와드리는 AI 어시스턴트입니다. 궁금한 점이 있거나 도움이 필요하신 것이 있으시면 편하게 말씀해 주세요. 무엇을 도와드릴까요?

[2] 사용자: 스파게티 맛있다
    AI: 네, 스파게티 정말 맛있죠! 🍝

스파게티는 정말 인기 있는 음식입니다. 여러 종류가 있어서 좋아요:

- **카르보나라** - 진하고 풍미 있는 맛
- **알리오 올리오** - 마늘과 올리브유의 간단하지만 깊은 맛
- **볼로네제** - 고기 소스의 풍부한 맛
- **해산물 스파게티** - 신선한 맛
- **크림 소스** - 부드럽고 고소한 맛

민수님은 어떤 종류의 스파게티를 가장 좋아하세요? 😋

[3] 사용자: 2+7 = 9.
    AI: 음, 계산을 다시 확인해 보세요! 😊

2 + 7 = **9**가 아니라 **9**입니다.

아, 잠깐! 다시 보니 2 + 7 = **9**가 맞네요!

제가 헷갈렸네요. 맞습니다! 2 + 7 = 9가 정답입니다. ✓

다른 계산이 필요하신가요?

[4] 사용자: 오 샹젤리제.
    AI: 아, "오 샹젤리제"! 🎵

정말 아름다운 노래네요! 프랑스의 유명한 곡이죠.

**"오 샹젤리제"** (Aux Champs-Élysées)는:
- 1969년에 발표된 유명한 프랑스 가요
- 가수 **Michel Fugain**이 부른 곡
- 파리의 샹젤리제 거리를 노래하는 로맨틱한 곡
- "♪ Aux Champs-Élysées ♪" 하는 후렴구가 정말 유명해요

파리의 샹젤리제는 실제로 매우 아름다운 거리로, 많은 사람들이 사랑하는 장소입니다. 🗼

이 노래를 좋아하시나요? 아니면 파리 여행을 계획 중이신가요? 😊

[5] 사용자: 날씨가 좋네.
    AI: 네, 정말 날씨가 좋네요! ☀️

오늘 날씨가 어떤가요?
- **맑고 화창한가요?** 🌞
- **따뜻한가요?*

### 5.2 저장된 상태 확인

In [ ]:
state = graph.get_state(config)

[HumanMessage(content='배가 많이 고프네', additional_kwargs={}, response_metadata={}),
 AIMessage(content='배가 고프시겠네요! 😋\n\n뭔가 먹고 싶으신 게 있나요?\n\n**어떤 음식이 땡기시나요?**\n- 🍜 국수나 국 같은 따뜻한 음식?\n- 🍔 버거나 샌드위치 같은 간단한 음식?\n- 🍱 밥과 반찬 같은 한끼 식사?\n- 🍕 피자나 튀김 같은 자극적인 음식?\n- 🍎 과일이나 간식?\n\n**혹은:**\n- 집에서 뭔가 만들어 먹을까요?\n- 외식을 할까요?\n- 배달을 시킬까요?\n\n편한 대로 선택하세요! 🙂 맛있게 드세요! 🍽️', additional_kwargs={}, response_metadata={'id': 'msg_017eKqPWwQfHcow8Hz72Ffou', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 720, 'output_tokens': 259, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--1138a82f-098b-498c-b044-ab7080cde8dd-0', usage_metadata={'input_tokens': 720, 'output_tokens': 259, 'total_tokens': 979, 'input_token_details':

In [ ]:

print(f"현재 저장된 메시지 수: {len(state.values['messages'])}")
print("\n저장된 대화:")
print("-" * 50)

for i, msg in enumerate(state.values['messages'], 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"[{i}] {role}: {content}")

현재 저장된 메시지 수: 6

저장된 대화:
--------------------------------------------------
[1] 사용자: 배가 많이 고프네
[2] AI: 배가 고프시겠네요! 😋

뭔가 먹고 싶으신 게 있나요?

**어떤 음식이 땡기시나요?**
...
[3] 사용자: 죽느냐 사느냐 그것이 문제로다.
[4] AI: 오, 셰익스피어의 **"햄릿"**에서 나온 유명한 명대사네요! 🎭

"To be, or n...
[5] 사용자: 내 이름이 뭐라고 했지?
[6] AI: 음, 저는 아직 당신의 **이름을 모르고 있어요!** 😊

우리 대화에서:
- 날씨 얘기 ...


---

## 6. 토큰 기반 Window 제한

메시지 개수 대신 토큰 수로 윈도우를 제한할 수도 있습니다.

In [ ]:
def count_tokens_simple(messages: list) -> int:
    """간단한 토큰 카운터 (실제로는 tiktoken 등 사용)"""
    total = 0
    for msg in messages:
        # 대략 4글자 = 1토큰 (한국어 기준 대략적 추정)
        total += len(msg.content) // 4
    return total


def trim_by_tokens(messages: list, max_tokens: int) -> list:
    """
    토큰 수 기준으로 메시지 트리밍.
    뒤에서부터 토큰 한도 내의 메시지만 유지.
    """
    if not messages:
        return messages

    result = []
    current_tokens = 0

    # 뒤에서부터 추가
    for msg in reversed(messages):
        msg_tokens = len(msg.content) // 4
        if current_tokens + msg_tokens <= max_tokens:
            result.insert(0, msg)
            current_tokens += msg_tokens
        else:
            break

    return result


# 테스트
test_messages = [
    HumanMessage(content="안녕하세요! 오늘 날씨가 좋네요 정말정말루요."),
    AIMessage(content="네, 정말 화창한 날씨입니다. 산책하기 좋겠어요."),
    HumanMessage(content="맛있는 점심 추천해주세요."),
    AIMessage(content="한식으로 비빔밥이나 불고기는 어떨까요?"),
    AIMessage(content="한식으로 비빔밥이나 불고기는 어떨까요?"),
    AIMessage(content="한식으로 비빔밥이나 불고기는 어떨까요?"),
    AIMessage(content="한식으로 비빔밥이나 불고기는 어떨까요?"),
    AIMessage(content="한식으로 비빔밥이나 불고기는 어떨까요?"),
]

print(f"전체 메시지: {len(test_messages)}개")
print(f"전체 토큰 (추정): {count_tokens_simple(test_messages)}개")

trimmed = trim_by_tokens(test_messages, max_tokens=30)
print(f"\n트리밍 후 (max 30토큰): {len(trimmed)}개")
print(f"트리밍 후 토큰: {count_tokens_simple(trimmed)}개")

전체 메시지: 8개
전체 토큰 (추정): 40개

트리밍 후 (max 30토큰): 6개
트리밍 후 토큰: 28개


---

## 7. Window Memory 특징

### 장점
- 토큰 비용 일정하게 유지
- 구현이 비교적 간단
- 최근 맥락은 유지

### 단점
- 오래된 중요 정보 손실
- 윈도우 크기 설정이 어려움

### 적합한 사용 사례
- 간단한 대화형 챗봇
- 최근 맥락만 중요한 경우
- 토큰 비용이 중요한 경우

### 다음 단계

**03_summary_memory.ipynb**에서는 대화를 요약하여 저장하는 Summary Memory를 배웁니다.